# Pandas进阶

上堂课留了个作业，关于『男女评价差异最悬殊的前十名电影』。

基本上跟那个『年龄差异悬殊』的case差不多，大家在助教课应该都已经实现了。

今天再来讲一下，如何更加简单粗暴地用另一种方法解决这个问题：

前面还是一样，导入库，清理数据

In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('data/ml-100k/u.user', sep='|', names=u_cols, encoding = "ISO-8859-1")

r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv('data/ml-100k/u.data', sep='\t', names=r_cols, encoding = "ISO-8859-1")

m_cols = ['movie_id', 'title', 'release_date', 'video_release_date', 'imdb_url']
movies = pd.read_csv('data/ml-100k/u.item', sep='|', names=m_cols, usecols=range(5), encoding = "ISO-8859-1")

movie_ratings = pd.merge(movies, ratings)
lens = pd.merge(movie_ratings, users)

我们用 `pivot_table（）` ：

我们先看一下原来的表长什么样子

In [29]:
lens.head()

,movie_id,title,release_date,video_release_date,imdb_url,user_id,rating,unix_timestamp,age,sex,occupation,zip_code
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,308,4,887736532,60,M,retired,95076
1,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,308,5,887737890,60,M,retired,95076
2,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),308,4,887739608,60,M,retired,95076
3,7,Twelve Monkeys (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Twelve%20Monk...,308,4,887738847,60,M,retired,95076
4,8,Babe (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Babe%20(1995),308,5,887736696,60,M,retired,95076


Pivot的功能就是一步到位的`group+unstack`

这个方法可以直接在参数中，录入你所需要『制作新表』的参数：

（当然，你要记得告诉pandas，你把这些数据agg起来的方法是啥。默认就是'mean', 你也可以使用std，等等其他统计方法）

此刻，相当于，一级groupby电影标题，二级groupby性别

In [30]:
pivoted = pd.pivot_table(lens, values='rating', index=['movie_id','title'],
...                     columns=['sex'], fill_value=0, aggfunc='mean')
pivoted.head()

,sex,F,M
movie_id,title,,
1,Toy Story (1995),3.789916,3.909910
2,GoldenEye (1995),3.368421,3.178571
3,Four Rooms (1995),2.687500,3.108108
4,Get Shorty (1995),3.400000,3.591463
5,Copycat (1995),3.772727,3.140625


这时候，我们把差值存成另一个column：diff

In [31]:
pivoted['diff']=pivoted.M - pivoted.F
pivoted.head()

,sex,F,M,diff
movie_id,title,,,
1,Toy Story (1995),3.789916,3.909910,0.119994
2,GoldenEye (1995),3.368421,3.178571,-0.189850
3,Four Rooms (1995),2.687500,3.108108,0.420608
4,Get Shorty (1995),3.400000,3.591463,0.191463
5,Copycat (1995),3.772727,3.140625,-0.632102


接下来我们直接排序diff即可。

最后，我们总结一眼Cheat_Sheet.

基本都被我们两堂课的内容给cover了。

同学们以后，可以留着这个CheatSheet，然后一时脑抽的时候，可以拿出来对照一下。